In [2]:
import pandas as pd
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [3]:
oush_df = pd.read_csv('./new_data.csv')
reuters_df = pd.read_csv('./reuters1000.csv', index_col=None).iloc[:,:5]
# reuters_df.to_csv('./reuters1000.csv',index=False)
# print('Finish')
train_size = 700
Y_train = oush_df.iloc[:train_size,1:].values
Y_test = oush_df.iloc[train_size:,1:].values

In [4]:
reuters_df.head(3)

,text,pl_usa,to_earn,to_acq,pl_uk
0,showers continued throughout the week in\nthe ...,1,0,0,0
1,standard oil co and bp north america\ninc said...,1,0,0,0
2,texas commerce bancshares inc's texas\ncommerc...,1,0,0,0


In [5]:
def cleanse(text):
    text = text.lower()
    filters = ['"','#','$','%','&','\(','\)','\*','\+',',','-','\.','/',':',';','<','=','>','\?','@','\[','\\','\]'
               ,'^','_',' ','\{','\|','\}','~','\t','\n','\x97','\x96']
    text = re.sub("<.*?>",' ',text)
    text = re.sub("|".join(filters)," ",text)
    return text

In [6]:
content = np.array(oush_df.iloc[:3,0])

In [7]:
content = np.array(oush_df.iloc[:,0])
total_size = len(content)
new_contents = []
document = []
for text in content:
    new_text = cleanse(text)
    split_content = word_tokenize(new_text)
    words = [word for word in split_content if word not in stopwords.words('english')]
    lemmed_word = [WordNetLemmatizer().lemmatize(w) for w in words]
    document.append(' '.join(lemmed_word))
    new_contents.append(lemmed_word)

print(new_contents[0:2])

[['evaluation', '99tcm', 'technegas', 'ventilation', 'scintigraphy', 'diagnosis', 'pulmonary', 'embolism', 'diagnosis', 'pulmonary', 'embolism', 'perfusion', 'lung', 'scintigraphy', 'offer', 'high', 'sensitivity', 'low', 'specificity', 'specificity', 'significantly', 'increased', 'use', 'combined', 'ventilation', 'perfusion', 'study', 'aspect', 'perfusion', 'lung', 'scintigraphy', 'uniformly', 'accepted', 'technique', 'ventilation', 'imaging', 'varies', 'centre', 'centre', 'study', 'describes', 'new', 'technique', 'performance', 'ventilation', 'scintigraphy', 'using', 'suspension', 'ultrafine', 'carbon', 'particle', 'labelled', '99tcm', 'technegas', 'technique', 'combine', 'ready', 'availability', '99tcm', 'optimal', 'imaging', 'property', 'easily', 'administered', 'radiopharmaceutical', 'particle', 'size', 'sufficiently', 'small', 'deposit', 'alveolus', '63', 'patient', 'studied', 'conventional', 'perfusion', 'scintigraphy', 'plus', 'technegas', 'ventilation', 'scintigraphy', 'image',

In [48]:
import collections
counter = collections.Counter([word for words in new_contents for word in words ])
# 'The count of elements not in the Counter is zero.'
        # Needed so that self[missing_item] does not raise KeyError
counter = dict(filter(lambda x: x[1] >= 300, counter.items()))
idx_to_token = [tk for tk, _ in counter.items()]
token_to_idx = {tk: idx for idx, tk in enumerate(idx_to_token)}

In [49]:
len(counter)

29

In [33]:
value=list(counter.values())
list.sort(value, reverse=True)

In [39]:
value[:int(0.1*len(value))][-1]

93

然后将词映射到整数索引。

In [50]:

# dataset = [[token_to_idx[tk] for tk in st if tk in token_to_idx]
#            for st in new_contents]
dataset = np.zeros(shape=[len(new_contents), len(idx_to_token)])
for i in range(len(new_contents)):
    for j in range(len(new_contents[i])):
        if new_contents[i][j] in token_to_idx:
            dataset[i,token_to_idx[new_contents[i][j]]] += 1
        # else:
        #     dataset[i,-1] += 1
# dataset = dataset / np.sum(dataset, axis=1, keepdims=True)
        
num_tokens = sum([len(st) for st in dataset])

D:\Softwares\miniconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [51]:
dataset[5]

array([0.05, 0.05, 0.  , 0.45, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.2 , 0.  , 0.  , 0.  ,
       0.  , 0.05, 0.1 , 0.05, 0.  , 0.  , 0.  ])

In [10]:
# vectorize = CountVectorizer(max_features=300,min_df=0.3)
# vectorize.fit(document)
# features = len(vectorize.vocabulary_)
# X_train = np.ones(shape=[train_size,features],dtype=np.int64)
# X_test = np.ones(shape=[total_size-train_size,features])
# for idx,new_content in enumerate(new_contents):
#     word_vector = vectorize.transform(new_content).toarray()
#     if idx >= train_size:
#         X_test[idx] = word_vector
#     else:
#          X_train[idx] = word_vector
# np.save('./oush_sample_train.npy',X_train)
# np.save('./oush_test.npy',X_test)
# np.save('./oush_label_train',Y_train)
# np.save('./oush_label_test',Y_test)
# print('Finish')